Nanopore DNA sequencing is portable and relatively cheap, allowing real time sequencing in the field.  We see the potential to use nanopore sequencing as an accessible educational experience. With a clear pipeline that Just Works(TM), a citizen scientist could do WIMP (What's in my pot?) analysis on their own samples without the need for any external tools.  Undergrad or high school students could follow the steps of the pipeline to learn about the basics of genome assembly.

# Why Nanopore is amazing
* Long reads, cheap, portable
* Comparison to the current standard (Illumina)
* Used for detecting ebola/zika?  Sent to space
* Sequencing in the jungle (tweet below!)
* Idea we care about: you should be able to take a random sample of stuff (ocean water?  dirt?) and sequence it cheaply and easily to find out what's there

In [9]:
%%html
<blockquote class="twitter-tweet" data-lang="en"><p lang="en" dir="ltr">Welcome to my laboratory :)<br><br>Sequencing long ribosomal cluster from plants, insects &amp; fungi in real-time in the Amazon rainforest. Within a few mins of <a href="https://twitter.com/nanopore?ref_src=twsrc%5Etfw">@nanopore</a> data generated, performed BLAST &amp; got correct hits! Dual indexing looks great for pooling many samples<a href="https://twitter.com/hashtag/junglegenomics?src=hash&amp;ref_src=twsrc%5Etfw">#junglegenomics</a> <a href="https://t.co/UQVjYfmU8U">pic.twitter.com/UQVjYfmU8U</a></p>&mdash; Aaron Pomerantz (@AaronPomerantz) <a href="https://twitter.com/AaronPomerantz/status/980873273348038656?ref_src=twsrc%5Etfw">April 2, 2018</a></blockquote>
<script async src="https://platform.twitter.com/widgets.js" charset="utf-8"></script>

# How Nanopore actually works
Insert a good paragraph description here:

My current understanding is just like, DNA strand pulled through a pore, current across pore changes depending on the nucleotide, you end up with an electrical signal in fast5 format that can be converted to a fasta file (could be fastq but no one trusts quality scores?)

Poretools would fit in well here, show what the raw signal looks like, run poretools, show the sequence we get out (something like the notebook here: http://nbviewer.jupyter.org/github/arq5x/poretools/blob/master/poretools/ipynb/test_run_report.ipynb)

## Poretools
The raw data we get from nanopore sequencing is in the fast5 format.  This is just a series of current values that were read across the pore as the DNA strand passed through it.

We are going to start by looking at this fast5 data, containing current values, and converting it to a fasta file that contains nucleotides. 

This poretools tutorial is adapted from here: http://nbviewer.jupyter.org/github/arq5x/poretools/blob/master/poretools/ipynb/test_run_report.ipynb

First we're going to find our fast5 files.  Our sample fast5 file is in the "data" folder, so we set the variable dataDirectory to "data/".  If you are using your own data, change dataDirectory to the path to your .fast5 files.

In [17]:
# dataDirectory is the path to our fast5 file.
# If you are using your own data, change dataDirectory to the path to your .fast5 files.
dataDirectory = 'data/'

# Print the number of fast5 files in the dataDirectory.
# Click the "Run" button at the top of this page to run this code.
!find $dataDirectory -maxdepth 1 -name "*.fast5" | wc -l

2


Poretools has a number of command line options.  Let's start with the stats command, which will give us some basic statistics about our reads.

In [26]:
# The -q option stops poretools outputting any warning messages.
!poretools stats -q $dataDirectory

/bin/sh: 1: poretools: not found


# Clustering reads before assembly?
Are we going to use Mash?  Does Canu just do this for us?

# Assembling our reads
Assuming that we get Canu to work, an explanation of running Canu goes here.

If not, minimap and then miniasm (or other way around?).

General idea: all vs. all meaning trying to align each read to all the others, end up with contigs (miniasm gave us one really long contig?)

For documentation stuff, it would be really good to put something about using IGV/looking at the actual read pileup here.

# What's in our sample?

BLAST on our contig(s) except that it hasn't actually worked yet.  Want an output that shows species.

MetaGeneMark or other way to show genes and read counts, some nice way to visualize/explore this information (I'm not convinced that this is the best software for this yet)

Displaying pileups/genes in genome browser at the end?